# Dependance

In [1]:
import sys,os
import datetime, time
import cx_Oracle
import pandas as pd
import numpy as np
import traceback
import logging

BASE_DIR = r"C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\UAT_Test"
sys.path.append(BASE_DIR)

from string import Template

from core import sdm_check
from core.configadmin import ConfigAdmin
from core.color_me import ColorMe
from core.outlook import send_email
from core import log_ctrl

In [2]:
# TODO:Config
config_path = os.path.join(BASE_DIR, 'conf')
config_name = 'setting.ini'
conf = ConfigAdmin(config_path, config_name)
# Template
D3_excel = os.path.join(conf.read_config('Cofig_Path','D3_Report'),f'XRM Campaign Report {time.strftime("%Y%m%d",time.localtime())}.xlsx')
D3_html_R = os.path.join(conf.read_config('Cofig_Path','D3_Report'),f'XRM Regular Campaign Report {time.strftime("%Y%m%d",time.localtime())}.html')
D3_html_O = os.path.join(conf.read_config('Cofig_Path','D3_Report'),f'XRM One Time Campaign Report {time.strftime("%Y%m%d",time.localtime())}.html')
MyTemp = os.path.join (config_path, 'CSS', conf.read_config('CSS_Style', 'XRMCampaignTemplate'))
MyCss_Style = os.path.join (config_path, 'CSS', conf.read_config('CSS_Style', 'table_style'))
# Log Info
log_info = log_ctrl.de8ug_log(logger_name='regular_check', log_file=os.path.join(BASE_DIR, 'log', f'{time.strftime("%Y%m",time.localtime())}-monitor.log'), level=logging.INFO)
# SQL Script
CampaignListWaitToCheck = os.path.join(BASE_DIR, 'db','sql','CampaignListWaitToCheck.sql')
CampaignAnaylysis = os.path.join(BASE_DIR, 'db','sql','CampaignAnaylysis.sql')
# SAS DB
user = conf.read_config("SAS_db", "user")
password = conf.read_config("SAS_db", "password")
service = conf.read_config("SAS_db", "service")
# panda option
a = lambda x: format(x, '.2%') if x != np.nan else ""
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
# pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
# pd.set_option('max_colwidth',100)
# pd.set_option("display.max_columns",18)
pd.set_option('display.width', 2000)    # 设置行宽
pd.set_option('colheader_justify', 'center')    #设置居中

# Scripts

## Memo

In [3]:
class regular_check(object):
    def __init__(self, db, sql, sender, copyer, temp):
        self.user = conf.read_config("SAS_db", f"{db}_user")
        self.password = conf.read_config("SAS_db", f"{db}_password")
        self.service = conf.read_config("SAS_db", f"{db}_service")
        self.db = cx_Oracle.connect(self.user, self.password, self.service)
        self.con = self.db.cursor()
        self.sql = sql
        self.notify = False
        self.sender = sender
        self.copyer = copyer
        self.temp = temp
    
    def excute(self):
        row = self.con.execute(sql).fetchall()
#         cols = [i[0] for i in self.con.description]
#         self.df = pd.DataFrame(data=row, index=[i[0] for i in row], columns=cols)
#         print(row)
        if row[0][0] == 0:
            self.Email()
            self.exit()
            return True
        else:
            self.exit()
            return False
        
    
    def Email(self):
        """
        Docstring:
        To sent the report.
        """
        subject = f"[{str(self.service).upper()}]-Data monitoring alert"
        content = open(self.temp, 'r', encoding='utf-8').read()
        if not send_email.outlook(send_to=self.sender, subject=subject, content=content, cc_to=self.copyer, addtach=None):
            logging.error('E-mail Send Faild!')
            print(ColorMe(f'{datetime.datetime.now().strftime("%Y_%m_%d %X")} E-mail Send Faild!').red())
            return False
        else:
            logging.info('E-mail Send Success!')
            print(ColorMe(f'{datetime.datetime.now().strftime("%Y_%m_%d %X")} E-mail Send Success!').green())
            return True
    
    def exit(self):
        self.db.close()

## Test

In [4]:
if __name__ == "__main__":
    trigger = False
    sqldir = os.path.join(BASE_DIR, 'db','sql','rtdm_decision_log_monitor.sql')
#     with open(sqldir, 'r', encoding='utf-8') as f:
#         sql = f.read()
    sender = 'CRMCampaignTeam@homecredit.cn'
    copyer = None
    temp = os.path.join (config_path, 'CSS', 'DecisionLogCNT0.html')
#     for i in range(2):
    with open(sqldir, 'r', encoding='utf-8') as f:
        Tempstr = f.read()
            
    switch = True
    print(datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'),'Program starting')
    try:
        while switch:
            if 8 <= int(datetime.datetime.strftime(datetime.datetime.now(), '%H')) < 21:
                sec = 10
            else:
                sec = 120
            sql = Template(Tempstr).substitute(sec=sec)
            trigger = regular_check('rtdm', sql, sender, copyer, temp).excute()
            if trigger == False:
    #             print(datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'), f'Normal status: wait {sec} sec')
                time.sleep(sec)
            else:
                print(datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'), 'Trigger alert: wait 15 min')
                time.sleep(900)
    #         i = 1 
    except Exception as e:
        print(datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'), e)
        content = traceback.format_exc()
        send_email.outlook(send_to="Leo.ZhangZS@homecredit.cn", subject="Program Error", content=content, cc_to=None, addtach=None)

2020-05-06 10:12:47 Program starting
2020_05_06 23:16:39 E-mail Send Success!
2020-05-06 23:16:39 Trigger alert: wait 15 min
2020_05_06 23:31:42 E-mail Send Success!
2020-05-06 23:31:42 Trigger alert: wait 15 min
2020_05_06 23:46:43 E-mail Send Success!
2020-05-06 23:46:44 Trigger alert: wait 15 min
2020_05_07 00:01:47 E-mail Send Success!
2020-05-07 00:01:47 Trigger alert: wait 15 min
2020_05_07 00:16:49 E-mail Send Success!
2020-05-07 00:16:50 Trigger alert: wait 15 min
2020_05_07 00:31:52 E-mail Send Success!
2020-05-07 00:31:52 Trigger alert: wait 15 min
2020_05_07 00:46:55 E-mail Send Success!
2020-05-07 00:46:55 Trigger alert: wait 15 min
2020_05_07 01:04:00 E-mail Send Success!
2020-05-07 01:04:01 Trigger alert: wait 15 min
2020_05_07 01:19:04 E-mail Send Success!
2020-05-07 01:19:04 Trigger alert: wait 15 min
2020_05_07 01:36:09 E-mail Send Success!
2020-05-07 01:36:09 Trigger alert: wait 15 min
2020_05_07 01:51:12 E-mail Send Success!
2020-05-07 01:51:12 Trigger alert: wait 15

KeyboardInterrupt: 